##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0


Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 1s 0us/step


Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[ 0.20030206, -0.08370684, -0.38809136,  0.2878713 ,  0.11109585,
         0.03716441,  0.31214017,  0.22107327,  0.21885161,  0.07126305]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.10868844, 0.08181631, 0.06034584, 0.11863538, 0.09941264,
        0.09232803, 0.12154975, 0.11096964, 0.11072338, 0.09553059]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.3824074

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [9]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 27s 455us/sample - loss: 0.2968 - accuracy: 0.9138
Epoch 2/5
60000/60000 [==============================] - 24s 399us/sample - loss: 0.1431 - accuracy: 0.9582
Epoch 3/5
60000/60000 [==============================] - 24s 393us/sample - loss: 0.1064 - accuracy: 0.9682
Epoch 4/5
60000/60000 [==============================] - 24s 404us/sample - loss: 0.0881 - accuracy: 0.9725
Epoch 5/5
60000/60000 [==============================] - 23s 390us/sample - loss: 0.0747 - accuracy: 0.9761


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

10000/1 - 3s - loss: 0.0389 - accuracy: 0.9756


[0.07742728491169401, 0.9756]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: id=30017, shape=(5, 10), dtype=float32, numpy=
array([[7.03256076e-09, 1.67420477e-08, 5.87022032e-06, 5.22389892e-04,
        6.85352441e-10, 1.03525217e-07, 6.51673637e-12, 9.99410510e-01,
        4.28912517e-06, 5.67821080e-05],
       [1.57161949e-07, 1.90219231e-04, 9.99640465e-01, 1.31990542e-04,
        3.00081683e-13, 3.67356988e-05, 1.89148395e-07, 5.54231742e-15,
        7.23882181e-08, 3.51056792e-12],
       [3.18723551e-07, 9.98535991e-01, 4.33777721e-04, 1.84476248e-05,
        8.25599454e-06, 3.37574875e-06, 4.22734684e-06, 6.56241493e-04,
        3.37720121e-04, 1.60965737e-06],
       [9.99909878e-01, 2.55149152e-10, 4.71661333e-05, 1.03160346e-07,
        4.98819439e-08, 8.17626642e-06, 2.90528351e-05, 1.64739845e-06,
        2.64704312e-08, 3.75646732e-06],
       [6.99663687e-08, 5.78081298e-11, 3.74255569e-06, 3.71527591e-08,
        9.98297274e-01, 9.56534834e-08, 7.44440456e-07, 6.63444007e-05,
        2.60770037e-07, 1.63140916e-03]], dtype=float32)>

# Let's Create a Challenger Model
for this model we will use a two layer approach, instead of just one layer and in order to help prevent signficant overfitting we will increase the dropout to 30%

In [18]:
challenger_model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(10)
])
challenger_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


Use the same loss function, metric and gradient descent optimizer

In [19]:
challenger_model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

Still train for five passes over the data

In [20]:
challenger_model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 28s 471us/sample - loss: 0.2731 - accuracy: 0.9192
Epoch 2/5
60000/60000 [==============================] - 25s 420us/sample - loss: 0.1154 - accuracy: 0.9654
Epoch 3/5
60000/60000 [==============================] - 25s 410us/sample - loss: 0.0842 - accuracy: 0.9742
Epoch 4/5
60000/60000 [==============================] - 25s 417us/sample - loss: 0.0665 - accuracy: 0.9796
Epoch 5/5
60000/60000 [==============================] - 24s 400us/sample - loss: 0.0543 - accuracy: 0.9829


With our larger model we were able to slightly increase accuracy (0.2%) on the validation set. The training time was actually about the same as the one layer model.

In [21]:
challenger_model.evaluate(x_test,  y_test, verbose=2)

10000/1 - 3s - loss: 0.0393 - accuracy: 0.9776


[0.07808237452211324, 0.9776]

# Convolution Model

For this model we look at a two stage convolutional neural network. We start with a kernel of size 5, and then neck down to a kernel of size 3 with a pooling layer after each stage. Finally we flatten into a fully connected layer prior to our output classification layer. We use fairly heavy regularization in the form of dropout to build a more robust network that generalizes better. The proof is in the results with our test set accuracy being nearly 99%.

In [30]:
conv_model = tf.keras.Sequential([
    tf.keras.layers.Reshape(
        target_shape=[28, 28, 1],
        input_shape=(28, 28,)),
    tf.keras.layers.Conv2D(16, 5, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dropout(rate=0.4),
    tf.keras.layers.Dense(10)])
conv_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 7, 7, 32)         

In [31]:
conv_model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [32]:
conv_model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 36s 598us/sample - loss: 0.5628 - accuracy: 0.8126
Epoch 2/5
60000/60000 [==============================] - 33s 553us/sample - loss: 0.2618 - accuracy: 0.9147
Epoch 3/5
60000/60000 [==============================] - 33s 550us/sample - loss: 0.2061 - accuracy: 0.9352
Epoch 4/5
60000/60000 [==============================] - 34s 566us/sample - loss: 0.1785 - accuracy: 0.9440
Epoch 5/5
60000/60000 [==============================] - 33s 546us/sample - loss: 0.1589 - accuracy: 0.9511


In [33]:
conv_model.evaluate(x_test,  y_test, verbose=2)

10000/1 - 4s - loss: 0.0180 - accuracy: 0.9892


[0.03569540517386049, 0.9892]